In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [3]:
def scatter_plot(x_: np.ndarray,
                 y_: np.ndarray,
                 name_x: str,
                 name_y: str,
                 ax=None):
    """
    scatter plot
    """
    data = pd.DataFrame(np.concatenate([x_, y_], axis=1),
                    columns=[name_x, name_y])
    if ax is None:
        sns.jointplot(data=data, x=name_x, y=name_y, kind="reg")
    else:
        sns.jointplot(data=data, x=name_x, y=name_y, kind="reg", ax=ax)

In [4]:
def performance_metrics_of_regression(labels: np.ndarray,
                                      predictions: np.ndarray) -> tuple:
    """
    Compute standard performance metrics for regression: mse and Rsquared
    """
    mse = ((predictions - labels) ** 2).mean()
    r2 = 1 - mse / (labels ** 2).mean()
    print(f'r2 = {"%.2f" % r2}, mse = {"%.2f" % mse}')

    return mse, r2

#We now experiment with random features ([Random Features for Large-Scale Kernel Machines](https://people.eecs.berkeley.edu/~brecht/papers/07.rah.rec.nips.pdf)) applied to the classic dataset from [A Comprehensive Look at the Empirical Performance of Equity Premium Prediction](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=517667) and implemented in [The Virtue of Complexity in Return Prediction](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3984925). We will also load the FRED-MD dataset from [FRED-MD](https://research.stlouisfed.org/wp/more/2015-012)

In [5]:
# AND NOW WE START WORKING WITH REAL DATA
# import os
# from google.colab import drive
# import pandas as pd
# drive.mount('/content/gdrive')
# folder = '/content/gdrive/My Drive/macro_data'

goyal_welch_data = pd.read_csv('../macro_data/GoyalWelchPredictorData2022Monthly.csv', index_col=0)
goyal_welch_data.index = pd.to_datetime(goyal_welch_data.index, format='%Y%m')

#fred_data = pd.read_csv(os.path.join(folder,'FRED_MD.csv'), index_col=0).iloc[1:, :]
#fred_data.index = pd.to_datetime(fred_data.index)

for column in goyal_welch_data.columns:
    goyal_welch_data[column] = [float(str(x).replace(',', '')) for x in goyal_welch_data[column]]


In [ ]:
goyal_welch_data

In [ ]:
goyal_welch_data['Index'].plot()

In [ ]:
np.log(goyal_welch_data['Index']).plot()

In [ ]:
goyal_welch_data.columns

In [ ]:
(12 * goyal_welch_data.Rfree).plot()

In [ ]:
goyal_welch_data.tail(20)


In [12]:
goyal_welch_data['returns'] = ((goyal_welch_data.Index)/ goyal_welch_data.Index.shift(1) - 1).fillna(0)

In [ ]:
goyal_welch_data['returns'].describe()


In [ ]:
goyal_welch_data.Rfree.describe()


#Excess returns are defined as $R_{t+1}-r_{f,t}$ (in excess of the risk-free rate). Then, we know that, with portfolio weight $\pi_t,$ the wealth evolves as
$$
W_{t+1}\ =\ W_t(r_{f,t} + \pi_t (R_{t+1}-r_{f,t}))
$$

In [ ]:
goyal_welch_data['excess_returns'] = goyal_welch_data.returns - goyal_welch_data.Rfree
leverage = 20.
strategy_returns = 1 + goyal_welch_data.Rfree + leverage * goyal_welch_data.excess_returns
plt.plot(np.cumprod(strategy_returns))
plt.title(f'leverage = {leverage}')

In [ ]:
goyal_welch_data.columns

In [ ]:
(12 * goyal_welch_data.infl).plot()

In [ ]:
goyal_welch_data.infl.describe()


In [ ]:
goyal_welch_data.tail(20)



In [ ]:
goyal_welch_data.corr()

In [ ]:
goyal_welch_data.dropna()

In [ ]:
goyal_welch_data


In [23]:
cleaned_data = goyal_welch_data.loc['1975':].drop(columns=['csp']).fillna(0)

In [ ]:
cleaned_data

In [ ]:
cleaned_data[['Index', 'D12', 'E12']] = (cleaned_data[['Index', 'D12', 'E12']] / cleaned_data[['Index', 'D12', 'E12']].shift(1)).fillna(0)
print(cleaned_data)

# **Do not forget to shift the signals !!!!**

In [ ]:
signal_columns = ['Index', 'D12', 'E12', 'b/m', 'tbl', 'AAA', 'BAA', 'lty', 'ntis',
        'Rfree', 'infl', 'ltr', 'corpr', 'svar']

data_for_signals = cleaned_data[signal_columns].shift(1).fillna(0) # shifting of signals happens here !
labels = cleaned_data.excess_returns.values.reshape(-1, 1)
data_for_signals['infl'] = data_for_signals['infl'].shift(1).fillna(0) # this is because inflation is actually published later
signals = data_for_signals.values
data_for_signals.shape, data_for_signals.columns

# Now comes (as usual) our favorite regression function
# Recall that we would like to compute $(zI+SS'/T)^{-1}$ for many values of $z.$ Doing this is slow because matrix inversion is slow. Instead, we will only pay the cost of matrix manipulations just once. We do this by performing the eigenvalue decomposition
$$
SS'/T\ =\ U DU'
$$
#and then use the mathematical formula
$$
(zI+SS'/T)^{-1}\ =\ U (zI+D)^{-1}U'\,.
$$
#The **estimated** regression coefficients for $y=S\beta+\epsilon$ are
$$
\hat\beta(z)\ =\ (zI+S'S/T)^{-1}S'y/T\ =\ S'(zI+SS'/T)^{-1}y/T
$$
#and we can thus rewrite it as
$$
S'(zI+SS'/T)^{-1}y/T\ =\ S'U (zI+D)^{-1}U' y/T\,.
$$
#So we proceed as follows. First, we compute
$$
\mu\ =\ U' y/T
$$
#Now, let $\delta$ be the vector of diagoal elements of $D$. Then,
$$
[(z_1I+D)^{-1}\mu, \cdots, (z_KI+D)^{-1}\mu]\ =\ [(z_1+\delta)^{-1}, \cdots, (z_K+\delta)^{-1}]*\mu
$$
#Here, $[(z_1+\delta)^{-1}, \cdots, (z_K+\delta)^{-1}]$ is the matrix with $K$ columns and $T$ rows, because $\mu$ and $\delta$ are $T$-dimensional.


In [27]:
def ridge_regr(signals: np.ndarray,
                labels: np.ndarray,
                future_signals: np.ndarray,
                shrinkage_list: np.ndarray):
    """
    Regression is
    beta = (zI + S'S/t)^{-1}S'y/t = S' (zI+SS'/t)^{-1}y/t
    Inverting matrices is costly, so we use eigenvalue decomposition:
    (zI+A)^{-1} = U (zI+D)^{-1} U' where UDU' = A is eigenvalue decomposition,
    and we use the fact that D @ B = (diag(D) * B) for diagonal D, which saves a lot of compute cost
    :param signals: S
    :param labels: y
    :param future_signals: out of sample y
    :param shrinkage_list: list of ridge parameters
    :return:
    """
    t_ = signals.shape[0]
    p_ = signals.shape[1]
    if p_ < t_:
        # this is standard regression
        eigenvalues, eigenvectors = np.linalg.eigh(signals.T @ signals / t_)
        means = signals.T @ labels.reshape(-1, 1) / t_
        multiplied = eigenvectors.T @ means
        intermed = np.concatenate([(1 / (eigenvalues.reshape(-1, 1) + z)) * multiplied for z in shrinkage_list],
                                  axis=1)
        betas = eigenvectors @ intermed
    else:
        # this is the weird over-parametrized regime
        eigenvalues, eigenvectors = np.linalg.eigh(signals @ signals.T / t_)
        means = labels.reshape(-1, 1) / t_
        multiplied = eigenvectors.T @ means # this is \mu

        # now we build [(z_1+\delta)^{-1}, \cdots, (z_K+\delta)^{-1}] * \mu
        intermed = np.concatenate([(1 / (eigenvalues.reshape(-1, 1) + z)) * multiplied for z in shrinkage_list],
                                  axis=1)

        tmp = eigenvectors.T @ signals # U.T @ S
        betas = tmp.T @ intermed # (S.T @ U) @ [(z_1+\delta)^{-1}, \cdots, (z_K+\delta)^{-1}] * \mu
    predictions = future_signals @ betas
    return betas, predictions

# Sometimes, data normalization can be important. We introduce a function to do it here

In [28]:
def normalize(data: np.ndarray,
              ready_normalization: dict = None,
              use_std: bool = False)->tuple:
  """

  """

  if ready_normalization is None:
      data_std = data.std(0)
      data_mean = data.mean(0)
      if use_std:
        data = (data - data_mean) / data_std # this is z-scoring of the data
      else:
        data_max = np.max(data, axis=0)
        data_min = np.min(data, axis=0)
  else:
      data_std = ready_normalization['std']
      data_mean = ready_normalization['mean']
      if use_std:
        data = (data - data_mean) / data_std # this is z-scoring of the data
      else:
        data_max = ready_normalization['max']
        data_min = ready_normalization['min']
  if not use_std:
    data = data - data_min
    data = data/(data_max - data_min)
    data = data - 0.5
  normalization = {'std': data_std,
                   'mean': data_mean,
                    'max': data_max,
                    'min': data_min}
  return data, normalization

In [ ]:
signals.shape

In [ ]:
normalize_raw_data = True
cheat_and_use_future_data = False  # set to True if you want to have
#our fun experiment to show how even know a bit about the future can drastically imprpve performance

shrinkage_list = [0.00000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

split = int(signals.shape[0] / 2)
train_labels = labels[:split]
test_labels = labels[split:]

if normalize_raw_data:
    signals[:split, :], normalization_based_on_train = normalize(signals[:split], use_std=False)
    # this is our fun experiment to show how even know a bit about the future can drastically imprpve performance
    if cheat_and_use_future_data:
      signals[split:, :] = normalize(signals[split:, :])[0]
    else:
      signals[split:, :] = normalize(signals[split:, :],
                                     ready_normalization=normalization_based_on_train)[0]

beta_estimate_using_train_sample, oos_predictions = ridge_regr(signals=signals[:split, :],
                                                                labels=train_labels,
                                                                future_signals=signals[split:, :],
                                                                shrinkage_list=shrinkage_list)

oos_predictions = pd.DataFrame(oos_predictions, index=cleaned_data.index[split:], columns = shrinkage_list)
print(oos_predictions)

In [ ]:
pd.DataFrame(beta_estimate_using_train_sample, columns=shrinkage_list, index=data_for_signals.columns)

# Now we compute managed returns $\pi_t(z) \cdot R_{t+1}.$ Because we are predicting the market, we are "timing the market" and hence we will also call them "market timing returns"

In [ ]:
market_timing_returns = oos_predictions * test_labels.reshape(-1, 1)
print(market_timing_returns) # we have one timing return for each value of z

In [33]:
def sharpe_ratio(x):
  # We are computing the ANNUALIZED SHARPE RATIO, hence we need to multiply by sqrt(12)
  return np.round(np.sqrt(12) * x.mean(0) / x.std(0), 2)

In [34]:
cleaned_data = pd.concat([cleaned_data, market_timing_returns], axis=1)

In [35]:
tmp = cleaned_data[['excess_returns'] + shrinkage_list].iloc[split:]
tmp = tmp / tmp.std()
sr = sharpe_ratio(tmp)


In [ ]:
tmp.cumsum().plot()
plt.title(f'SR={sr.values.flatten()}')

# We now investigate the statistical significance of our strategy performance relative to that of the two benchmarks: the market and the simple, linear strategy. To this end, we run the following regression
$$
R_{t+1}^{simple\ linear}\ =\ \alpha\ +\ \beta R_{t+1}^{market}
$$
#We would like to see a large positive t-statistic for the $\alpha$ coefficient

In [37]:
import statsmodels.api as sm

def regression_with_tstats(predicted_variable, explanatory_variables):
    x_ = explanatory_variables
    x_ = sm.add_constant(x_)
    y_ = predicted_variable
    # Newey-West standard errors with maxlags
    z_ = x_.copy().astype(float)
    result = sm.OLS(y_.values, z_.values).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
    try:
        tstat = np.round(result.summary2().tables[1]['z'], 1)  # alpha t-stat (because for 'const')
        tstat.index = list(z_.columns)
    except:
        print(f'something is wrong for t-stats')
    return tstat

In [ ]:
tstat = regression_with_tstats(predicted_variable=tmp[1.], explanatory_variables=tmp['excess_returns'])
print(tstat)

In [ ]:
tmp[1.]

# **NOW COME THE RANDOM FEATURES**

#The scale of random weights entering random features is very important.

#There are two equivalent ways to control the scale of your linear random features. One is to generate $\omega\sim N(0,1)$ and then use $\gamma\cdot \omega$ for some $\gamma\in \mathbb R$. This $\gamma$ controls the scale of the your features. Another way is to directly sample $\omega\sim N(0, \gamma^2).$ Mathematically, this is based on the important observation that if $\omega\sim N(0,1)$, then $\gamma\omega\sim N(0, \gamma^2).$

#The intuition is based on the Taylor approximation.  We have $h(\gamma \cdot \omega'x) \approx h(0)+h'(0) \gamma \omega' x$ when $\gamma$ is small enough. That is, our non-linear random features become approximately linear. **Thus, the bigger $\gamma$ is, the more non-linear the features are.**

#**This scale is also important for neural networks, and is pinned down through initialization!**

In [40]:
P = 50000
d = 14
scale = 1.
omega = scale * np.sqrt(2) * np.random.randn(d, P) / np.sqrt(d)
ins_sin = np.sqrt(2) * np.sin(signals @ omega) # this is n times P
ins_cos = np.sqrt(2) * np.cos(signals @ omega) # this is also n times P
random_features = np.append(ins_sin, ins_cos, axis=1) # this is n times (2P)

In [ ]:
random_features.shape


In [ ]:
split = int(signals.shape[0] / 2)

labels = cleaned_data.excess_returns.values.reshape(-1, 1)

train_labels = labels[:split]
test_labels = labels[split:]

beta_estimate_using_train_sample, oos_predictions = ridge_regr(signals=random_features[:split, :],
                                                                labels=train_labels,
                                                                future_signals=random_features[split:, :],
                                                                shrinkage_list=shrinkage_list)

oos_predictions = pd.DataFrame(oos_predictions, index=cleaned_data.index[split:], columns = shrinkage_list)
print(oos_predictions)

In [ ]:
market_timing_returns_complex = oos_predictions * test_labels
market_timing_returns_complex.columns = [f'{x}_complex' for x in market_timing_returns_complex.columns]

cleaned_data = pd.concat([cleaned_data, market_timing_returns_complex], axis=1)

# 'excess_returns' are just market returns; it is important we keep them
# shrinkage_list: these are the columns corresponding to the simple linear model with just 13 predictors
tmp = cleaned_data[['excess_returns'] + shrinkage_list + list(market_timing_returns_complex.columns)].iloc[split:, :]
tmp = tmp / tmp.std()
tmp['mean'] = tmp.mean(1)
sr = sharpe_ratio(tmp)
tmp.cumsum().plot()
plt.title(f'sr={sr}')
plt.savefig(os.path.join(folder, 'performance_pl9ot.jpeg'))



In [ ]:
cleaned_data.corr()[0.001]


In [49]:
import statsmodels.api as sm

def regression_with_tstats(predicted_variable, explanatory_variables):
    x_ = explanatory_variables
    x_ = sm.add_constant(x_)
    y_ = predicted_variable
    # Newey-West standard errors with maxlags
    z_ = x_.copy().astype(float)
    result = sm.OLS(y_.values, z_.values).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
    try:
        tstat = np.round(result.summary2().tables[1]['z'], 1)  # alpha t-stat (because for 'const')
        tstat.index = list(z_.columns)
    except:
        print(f'something is wrong for t-stats')
    return tstat

# We now investigate the statistical significance of our strategy performance relative to that of the two benchmarks: the market and the simple, linear strategy. To this end, we run the following regression
$$
R_{t+1}^{complex}\ =\ \alpha\ +\ \beta_1 R_{t+1}^{market}\ +\ \beta_2 R_{t+1}^{simple\ linear}
$$
#We would like to see a large positive t-statistic for the $\alpha$ coefficient

In [ ]:
tstats = regression_with_tstats(predicted_variable=tmp['0.001_complex'], explanatory_variables=tmp[['excess_returns', 0.001]])
print(tstats)

# SPY ticker 

In [ ]:
%run ./Macro_Assets.ipynb

In [ ]:
# Store a copy of the original data before modifying it
original_spy_data_full = spy_data_full.copy()
spy_data_full

In [ ]:
# Calculate monthly returns
spy_data_full['returns'] = spy_data_full['Close'].pct_change().fillna(0)

# Merge with the risk-free rate 
spy_data_full = spy_data_full.join(cleaned_data[['Rfree']], how='left')

# Excess returns
spy_data_full['excess_returns'] = spy_data_full['returns'] - spy_data_full['Rfree']

spy_data_full.head()


In [ ]:
# Signal columns 
SPY_df_signals = original_spy_data_full.join(data_for_signals[signal_columns], how='left')

SPY_df_signals
